In [1]:
from Bubble_dynamics_simulation import full_bubble_model as de
from Bubble_dynamics_simulation import excitation
from Bubble_dynamics_simulation import inp_data_extractor as inp
from Bubble_dynamics_simulation import data
from Bubble_dynamics_simulation import gradient_descent as gd
import parameters as par
import numpy as np
import importlib

model: chem_Otomo2018_without_O
target specie: NH3
excitation: sin_impulse (control parameters: ['p_A', 'freq', 'n'])
enable heat transfer: True	enable evaporation: False	enable reactions: True	enable dissipated energy: True


In [2]:
from termcolor import colored

In [25]:
def print_variable(name, value, use_type=False):
    """Print the variable name and value"""
    var_type = str(getattr(value, '__class__', '\'None\'')).split("'")[-2]
    if len(str(value)) < 80:
        value = str(value)
    else:
        value = str(value)[:80].replace('\n', ', ') + '...'
    if name == 'self':
        if use_type:
            return name
        return colored(name, 'blue')
    if type(value) == str:
        value = "'" + value + "'"
    if use_type:
        return f'{name}: ' + colored(var_type, 'blue') + ' = ' + colored(value, 'green')
    if var_type == 'NoneType':
        return colored(name, 'blue')
    return colored(name, 'blue') + '=' + colored(value, 'green')

print(print_variable('model', par.model, use_type=True))

model: str = 'chem_Otomo2018_without_O'


In [26]:
def print_function(fun):
    function_name = getattr(fun, '__name__', 'None')
    text = f'{function_name}('
    if type(fun) == type:
        fun = fun.__init__
    defaults = getattr(fun, '__defaults__', None)

    if defaults is not None:
        defaults = list(defaults)
    else:
        defaults = []

    if hasattr(fun, '__code__'):
            args = fun.__code__.co_varnames[:fun.__code__.co_argcount]
    else:
        args = []
    defaults = [None]*(len(args)-len(defaults)) + defaults
    for arg, default in zip(args, defaults):
        text += print_variable(arg, default) + ', '
    if len(args) > 0:
        text = text[:-2]
    return text + ')'

print(print_function(de.Make_dir))

Make_dir(self, folder_name, file_base_name='output_', separator=',')


In [27]:
def print_help(fun, indent=0):
    if not hasattr(fun, '__doc__') or fun.__doc__ is None:
        return ''
    text = fun.__doc__
    lines = text.split('\n')
    if text.startswith('_'):
        return ''
    if len(lines) < 2:
        return colored(indent*' ' + text.lstrip(), 'yellow', attrs=['dark'])
    spaces = len(lines[1]) - len(lines[1].lstrip())
    if lines[1].lstrip().startswith('*'):
        spaces -= 1

    text = ''
    for i, line in enumerate(lines):
        if (i == 0 or i == len(lines)-1) and line.lstrip() == '':
            continue
        line = line.removeprefix(spaces*' ')
        text += indent*' ' + line + '\n'
    
    return colored(text[:-1], 'yellow', attrs=['dark'])

print(print_help(gd._central_difference, indent=0))

Calculate the normed gradient of a point with central difference. Arguments:
 * point: dict, cpar
 * ranges: dict, ranges of the parameters ([single_value] or [min, max])
 * to_optimize: str, name of the output we want to optimize (e.g. 'energy_efficiency')
 * delta: float, step size for the finite difference
 * t_int, LSODA_timeout, Radau_timeout: see de.solve() for more info
 * verbose: bool, print stuff

Returns:
 * gradient: dict, normed gradient vector
 * datas: list of dicts, simulation results from de.get_data()


In [28]:
def list_functions(module, show_private=False, recursive=False, help=False):
    print(colored(f'\n{module.__name__}', attrs=['bold']))
    print('-'*len(module.__name__))
    print(print_help(module, indent=0))
    variables = []
    functions = []
    classes = []
    modules = []
    for i, element_name in enumerate(dir(module)):
        if element_name.startswith('__'):   # attribute
            continue
        if not show_private and element_name.startswith('_'):   # private
            continue
        element = getattr(module, element_name)

        if hasattr(element, '__package__'): # module (os, numpy, etc.)
            modules.append(element_name)
            continue
        if hasattr(element, '__module__'):  # class or function
            if module.__name__ != element.__module__:   # different module
                if not element.__module__.split('.')[0] == module.__package__:    # not the same package
                    continue

            if element.__class__ == type:   # class
                classes.append(element_name)
                continue
            else:   # function
                functions.append(element_name)
                continue

        else:   # variable or other
            if not callable(element):
                variables.append(element_name)
                continue
            else:   # other
                functions.append(element_name)
                continue

    # MODULES
    if len(modules) > 0:
        print(colored('Modules:', attrs=['bold']))
    for i, module_name in enumerate(modules):
        print(f'{i: >5}. {module_name}')
        element = getattr(module, module_name)
        if module.__package__ == element.__package__:
            if help and hasattr(element, '__doc__') and element.__doc__ is not None:
                print(print_help(element, indent=8))

    # CLASSES
    if len(classes) > 0:
        print(colored('Classes:', attrs=['bold']))
    for i, class_name in enumerate(classes):
        element = getattr(module, class_name)
        print(f'{i: >5}. {print_function(element)}')
        if help and hasattr(element, '__doc__') and element.__doc__ is not None:
            print(print_help(element, indent=8))
        for sub_element_name in dir(element):
            if sub_element_name.startswith('__'):
                continue
            if not show_private and sub_element_name.startswith('_'):
                continue
            sub_element = getattr(element, sub_element_name)
            print('        * ' + print_function(sub_element))
            if help and hasattr(sub_element, '__doc__') and sub_element.__doc__ is not None:
                print(print_help(sub_element, indent=12))

    # FUNCTIONS
    if len(functions) > 0:
        print(colored('Functions:', attrs=['bold']))
    for i, function_name in enumerate(functions):
        element = getattr(module, function_name)
        print(f'{i: >5}. {print_function(element)}')

        if help and hasattr(element, '__doc__') and element.__doc__ is not None:
            print(print_help(element, indent=8))

    # VARIABLES
    if len(variables) > 0:
        print(colored('Variables:', attrs=['bold']))
    for i, variable_name in enumerate(variables):
        variable = getattr(module, variable_name)
        print(f'{i: >5}. {print_variable(variable_name, variable, use_type=True)}')
        element_type = str(element.__class__).split("'")[-2]

    # Recursion (printing submodules)
    if recursive:
        for i, module_name in enumerate(modules):
            element = getattr(module, module_name)
            if module.__package__ == element.__package__:
                list_functions(element, show_private, recursive, help)



list_functions(de, show_private=False, recursive=False, help=False)


Bubble_dynamics_simulation.full_bubble_model
--------------------------------------------

Modules:
    0. excitation
    1. importlib
    2. np
    3. os
    4. par
    5. plt
    6. psutil
    7. socket
    8. time
Classes:
    0. Make_dir(self, folder_name, file_base_name='output_', separator=',')
        * close(self)
        * new_file(self)
        * write_line(self, data)
        * write_solution(self, data, num_sol, file_base_name)
        * write_string(self, string, file_base_name)
    1. dotdict()
        * clear()
        * copy()
        * fromkeys()
        * get()
        * items()
        * keys()
        * pop()
        * popitem()
        * setdefault()
        * update()
        * values()
Functions:
    0. Excitation(t, P_amb, args)
    1. VapourPressure(T)
    2. Viscosity(T)
    3. copy(input)
    4. example_cpar(normal_dict='False')
    5. get_data(cpar, num_sol, error_code, elapsed_time)
    6. get_errors(error_code, printit='False')
    7. get_settings_and_inf

In [29]:
list_functions(gd, show_private=True, recursive=False, help=True)


Bubble_dynamics_simulation.gradient_descent
-------------------------------------------

Modules:
    0. de

    1. importlib
    2. np
    3. random
    4. time
Functions:
    0. _central_difference(point, ranges, to_optimize='energy_efficiency', delta='1e-06', t_int='[0. 1.]', LSODA_timeout='30', Radau_timeout='300', verbose='True')
        Calculate the normed gradient of a point with central difference. Arguments:
         * point: dict, cpar
         * ranges: dict, ranges of the parameters ([single_value] or [min, max])
         * to_optimize: str, name of the output we want to optimize (e.g. 'energy_efficiency')
         * delta: float, step size for the finite difference
         * t_int, LSODA_timeout, Radau_timeout: see de.solve() for more info
         * verbose: bool, print stuff
        
        Returns:
         * gradient: dict, normed gradient vector
         * datas: list of dicts, simulation results from de.get_data()
    1. _forward_difference(point, ranges, to_opti